<a href="https://colab.research.google.com/github/osahonokoro/ammonia-sensor-simulation/blob/main/ammonia_sensor_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Install required libraries
!pip install pandas matplotlib seaborn

# 🧬 Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/osahonokoro/ammonia-sensor-simulation.git

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 📁 Set paths
csv_path = 'osahonokoro/ammonia-sensor-simulation/packet_log.csv'
output_dir = 'chunk_outputs'
os.makedirs(output_dir, exist_ok=True)

# 🔧 Parameters
chunk_size = 10000
ma_window = 50
all_chunks = []
alert_durations = []
normal_durations = []

# 📊 Process chunks
for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunk_size)):
    chunk['Timestamp'] = pd.to_datetime(chunk['Timestamp'])
    chunk.sort_values('Timestamp', inplace=True)
    chunk['NH4_Smoothed'] = chunk['NH4_Concentration'].rolling(ma_window, min_periods=1).mean()

    # Duration calculations
    alert_time = chunk[chunk['ALERT_Status'] == 'ALERT']['Timestamp']
    normal_time = chunk[chunk['ALERT_Status'] == 'NORMAL']['Timestamp']
    alert_durations.append((alert_time.max() - alert_time.min()).total_seconds() if not alert_time.empty else 0)
    normal_durations.append((normal_time.max() - normal_time.min()).total_seconds() if not normal_time.empty else 0)

    all_chunks.append(chunk)

    # 📈 Plot chunk trends
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=chunk, x='Timestamp', y='NH4_Concentration', label='Raw')
    sns.lineplot(data=chunk, x='Timestamp', y='NH4_Smoothed', label='Smoothed')
    plt.title(f'Chunk {i+1} NH₄⁺ Trend')
    plt.xlabel('Time')
    plt.ylabel('Concentration (mg/L)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{output_dir}/chunk_{i+1}_trend.png')
    plt.close()

# 🔗 Combine all chunks
full_data = pd.concat(all_chunks)

# 📊 Final summary plot
plt.figure(figsize=(14, 6))
sns.lineplot(data=full_data, x='Timestamp', y='NH4_Concentration', label='Raw')
sns.lineplot(data=full_data, x='Timestamp', y='NH4_Smoothed', label='Smoothed')
plt.title('Overall NH₄⁺ Concentration Trend')
plt.xlabel('Time')
plt.ylabel('Concentration (mg/L)')
plt.legend()
plt.tight_layout()
plt.savefig(f'{output_dir}/final_summary.png')
plt.close()

# 📋 Print summary stats
print("✅ Final Summary:")
print(f"Total ALERT duration: {sum(alert_durations)} seconds")
print(f"Total NORMAL duration: {sum(normal_durations)} seconds")
